# Translation

## 학습 목표

1. 다양한 RNN의 구성을 알아보기
2. 인코더와 디코더 구조의 필요성 이해하기
3. 교사 강요(teacher forcing)의 원리 알기
4. 훈련 단계와 추론 단계(inference)의 차이 알기

## 기계 번역 이야기

<img src = 'data/MT.png' height = '50%' width = '50%'>

* 규칙 기반 -> 통계적 기계 번역 -> 신경망 기계 번역
* seq2seq : 초창기 신경망 기계번역

## 시퀀스 처리하는 RNN

<img src = 'data/RNN.jpg' height = '50%' width = '50%'>

번역기는 첫번재 many to many 형식으로 만들어야 한다.

## seq2seq

### Encoder Decoder

<img src = 'data/seq2seq_1.jpg' height = '50%' width = '50%'>

* 2개의 RNN 아키텍쳐를 연결한 구조.
* 입력 문장을 받는 RNN을 인코더
* 두번째 RNN을 디코터

<img src = 'data/EncoderDecoder_1.png' height = '70%' width = '70%'>

* Encoder : Feature extractor 역할 - 어떤 데이터를 해석하기 위해 feature vector z를 만들어냅니다.
* Decoder : Feature z로부터 정보를 복원하여 다른 데이터를 재생성하는 역할.

* seq2seq 모델은 이 Encoder와 Decoder 둘 다 RNN 인 경우.
* feature vector : RNN이 문장을 해석해 만든 hidden state vector
* 문장 X를 z 라는 hidden state로 해석 후 z를 다시 언어 문장 Y롤 재생성
* 인코더에서 필요한 것은 인코더의 마지막 time step의 hidden state, 이를 두번째 RNN인 Decoder에게 전달

* 디코더는 인코더의 마지막 time step의 hidden state를 받아서 자신의 초기 hidden state로 하고 출력 문장을 생성.
* 여기서 특수 문자를 사용해서 출력 문장의 시작과 끝을 알려주기.
* 위 위 그림에서 GO, EOS 가 시작과 끝, 경우에 따라 SOS, EOS 라고도 함.

### Conditional Language Model

* 문장 생성기(Text Generator) : 언어 모델을 구현한 것
* 언어 모델 : n-1개의 단어 시퀀스 $w_1, ..., w_{n-1}$ 가 주어질때 n 번째 단어로 무엇이 올지 예측하는 확률 모델
* 파라미터 $\theta$를 모델링하는 언어 모델을 다음과 같이 표현
$$ P(w_n|w_1, ..., w_{n-1};\theta)$$
* 문장 생성기는 어떤 말을 만들고 싶은지 제어할 수 없다.


* 상황에 맞게 제어할 수 있게 언어모델을 확장하여 조건적 언어모델을 만듬.
$$ P(w_n|w_1, ..., w_{n-1},c;\theta)$$
* c : 아무 문장이나 만들지 말고 c에 맞는 문장을 만들어라.
* 인코더가 문장 X를 해석해서 *c* 를 만들고 *c* 를 문장생성기인 디코더에 입력으로 넣어주는 모델이 seq2seq

## 교사 강요(teacher forcing)

<img src = 'data/Teacher_forcing.png' height = '10%' width = '30%'>

* 훈련 과정에서 이전 time step이 잘못 예측하면 이번 예측도 잘못될 수 있고 이러면 훈련 시간이 엄청 길어진다.
* 이를 막기위해 이전 time step의 실제 값을 입력 값으로 사용하게 함.
* 이를 Teacher Forcing이라 한다.

## 단어 수준 VS 문자 수준

* Word level VS Character level
* time step의 입출력 단위
* 단어 집합은 엄청 많아지는 반면 문자는 알파벳 전부기 때문에 실제 구현 자체는 문자 수준이 더 편하다.
* 띄어쓰기 또는 많은 변종이 있는 언어는 문자 수준이 좋다.
* 하지만 그만큼 단어를 문자로 쪼개면서 데이터를 손실시키는것이기 때문에 충분한 데이터가 없다면 단어모델도 더 품질이 떨어진다.
* 최신 자연어처리는 subword를 기반으로 이루어진다.
* 우리는 문자 수준을 해보자.

# 번역기를 만들어보자

## 전처리

### Import Modules(library)

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np


#### GPU

In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(tf.config.list_physical_devices('GPU'))
device_lib.list_local_devices()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-08-22 19:14:18.908392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-22 19:14:18.978654: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-22 19:14:18.978900: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2467131201188759317
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6992625664
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 813260309171879551
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

2022-08-22 19:14:18.981449: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-22 19:14:18.982822: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-22 19:14:18.983142: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-22 19:14:18.983269: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

### 파일 불러오기

In [3]:
file_path = 'data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 194513


,eng,fra,cc
117054,Mary is a very attractive girl.,Mary est une fille très attirante.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
127775,I don't know how to speak French.,Je ne sais pas parler français.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
80472,Let's go outside and play.,Sortons dehors jouer.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
50925,Maybe I'll buy a bike.,J'achèterai peut-être un vélo.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
118576,Tom is at the neighbor's house.,Tom est chez le voisin.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


### CC 제거 및 5만개만 사용

In [4]:
lines = lines[['eng', 'fra']][:50000]
lines.sample(5)

,eng,fra
10185,How tall is he?,Quelle taille fait-il ?
46156,What's Tom's address?,Quelle est l'adresse de Tom ?
27160,What is your plan?,Quel est ton plan ?
49002,I find you attractive.,Je te trouve attirante.
33223,Who can prevent it?,Qui peut l'empêcher ?


### 시작과 종료 토큰 추가

In [16]:
func = lambda x : [0] + x
x= [5]
func(x)


[0, 5]

In [5]:
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 50000


,eng,fra
2895,Don't worry.,\t Ne vous en faites pas. \n
38388,Tom can't even read.,\t Tom ne peut même pas lire. \n
18244,I am a professor.,\t Je suis professeur. \n
598,I did OK.,\t Je m'en suis bien sorti. \n
38802,Tom was never happy.,\t Tom n'était jamais heureux. \n


### 단어장 및 정수 인코딩 과정

* char_level = True : 문자단위로 Tokenizer 생성
* eng, fra의 각행에 토큰화를 수행
* 단어를 숫자값 인덱스로 변환하여 저장

In [6]:
eng_tokenizer = Tokenizer(char_level=True) 
eng_tokenizer.fit_on_texts(lines.eng)
input_text = eng_tokenizer.texts_to_sequences(lines.eng)
input_text[:3]

[[19, 3, 8], [19, 3, 8], [19, 3, 8]]

In [7]:
fra_tokenizer = Tokenizer(char_level=True)
fra_tokenizer.fit_on_texts(lines.fra)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)
target_text[:3]

[[10, 1, 19, 5, 1, 31, 1, 11],
 [10, 1, 15, 5, 12, 16, 29, 2, 14, 1, 11],
 [10, 1, 26, 9, 8, 28, 2, 1, 31, 1, 11]]

* 단어장 크기를 변수로 저장. 0번 토큰을 생각해서 +1

In [8]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 53
프랑스어 단어장의 크기 : 73


### 최대 길이 구해보기

In [9]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 22
프랑스어 시퀀스의 최대 길이 76


### 통계 정보

In [10]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 50000
영어 단어장의 크기 : 53
프랑스어 단어장의 크기 : 73
영어 시퀀스의 최대 길이 22
프랑스어 시퀀스의 최대 길이 76


### 인코더와 디코더의 데이터

* 인코더 입력인 영어와 달리 프랑스 시퀀스는 2가지 버전으로 나누어 준비
* 하나의 디코더 출력과 비교해야 할 정답 데이터로 사용할 원래 목적
* 나머지 하나는 teacher forcing 용(디코더 입력용)

* 디코더 입력은 <'eos'>가 필요없고 출력과 비교할 시퀀스는 <'sos'>가 필요없다.
* A를 B로 번역할때 훈련과정에서 디코더가 <'sos'>A 를 받아서 B<'eos'> 를 예측하도록 훈련

In [11]:
encoder_input = input_text
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [12]:
print(decoder_input[:3])
print(decoder_target[:3])

[[10, 1, 19, 5, 1, 31, 1], [10, 1, 15, 5, 12, 16, 29, 2, 14, 1], [10, 1, 26, 9, 8, 28, 2, 1, 31, 1]]
[[1, 19, 5, 1, 31, 1, 11], [1, 15, 5, 12, 16, 29, 2, 14, 1, 11], [1, 26, 9, 8, 28, 2, 1, 31, 1, 11]]


### Padding 진행

In [13]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 22)
프랑스어 입력데이터의 크기(shape) : (50000, 76)
프랑스어 출력데이터의 크기(shape) : (50000, 76)


In [14]:
print(encoder_input[0])

[19  3  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


역시 0으로 패딩이 되었습니다.

### 이제 정수에 따른 벡터화 방법으로 one-hot encoding 사용

* 샘플의 크기 = 샘플의 수 * 샘플의 크기 * 단어장의 크기

In [15]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 22, 53)
프랑스어 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 출력데이터의 크기(shape) : (50000, 76, 73)


### Validation 데이터 분리

* 검증을 위해 5만 건 중에 3천건만 검증 데이터로 삼고, 나머지를 학습

In [16]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (50000, 22, 53)
프랑스어 학습 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 학습 출력데이터의 크기(shape) : (50000, 76, 73)


## 모델 훈련

[참고](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)

### 추가 라이브러리 불러오기

In [17]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

### 인코더 설계 

* LSTM 설계
* LSTM 은 hidden state, cell state 2개 존재
* Encoder LSTM 이 마지막 time step의 hidden state, cell state 를 Decoder LSTM 의 첫 번째 hidden state, cell state 로 전달

* 입력 텐서 생성.
* hidden size가 256인 인코더의 LSTM 셀 생성, return_state = True 를 통해 hidden state, cell state 를 return
* 디코더로 전달할 마지막 time step의 hidden state, cell state를 리턴. 
* hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.

In [18]:
encoder_inputs = Input(shape=(None, eng_vocab_size))
encoder_lstm = LSTM(units = 256, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

2022-08-22 19:14:23.746800: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-22 19:14:23.746989: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-22 19:14:23.747094: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-22 19:14:23.747236: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-22 19:14:23.747366: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

### 디코더 설계 

* 입력 텐서 생성.
* hidden size가 256인 인코더의 LSTM 셀 생성
* decoder_outputs는 모든 time step의 hidden state
* initial_state 를 통해 초기 상태를 지정가능(이전의 encoder_states 사용)

In [19]:
decoder_inputs = Input(shape=(None, fra_vocab_size))
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

### 디코더 출력층

* dense layer 에 softmax로 단어장에서 1개만 선택(문자 기준이므로)

In [20]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

### 하나의 모델 만들기

* 인코더와 디코더를 연결

In [21]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 53)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 73)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        317440      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

### 모델 훈련

In [22]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

2022-08-22 19:14:24.647095: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1043024000 exceeds 10% of free system memory.
2022-08-22 19:14:25.489727: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1043024000 exceeds 10% of free system memory.
2022-08-22 19:14:26.387027: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1043024000 exceeds 10% of free system memory.
2022-08-22 19:14:26.919331: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1043024000 exceeds 10% of free system memory.


Epoch 1/50


2022-08-22 19:14:31.093168: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


368/368 [==============================] - 11s 17ms/step - loss: 0.8860 - val_loss: 0.7733
Epoch 2/50
368/368 [==============================] - 6s 16ms/step - loss: 0.5443 - val_loss: 0.6348
Epoch 3/50
368/368 [==============================] - 6s 17ms/step - loss: 0.4514 - val_loss: 0.5616
Epoch 4/50
368/368 [==============================] - 6s 17ms/step - loss: 0.3971 - val_loss: 0.5122
Epoch 5/50
368/368 [==============================] - 6s 16ms/step - loss: 0.3602 - val_loss: 0.4685
Epoch 6/50
368/368 [==============================] - 6s 17ms/step - loss: 0.3339 - val_loss: 0.4433
Epoch 7/50
368/368 [==============================] - 6s 17ms/step - loss: 0.3142 - val_loss: 0.4277
Epoch 8/50
368/368 [==============================] - 6s 17ms/step - loss: 0.2983 - val_loss: 0.4097
Epoch 9/50
368/368 [==============================] - 6s 17ms/step - loss: 0.2851 - val_loss: 0.3983
Epoch 10/50
368/368 [==============================] - 6s 16ms/step - loss: 0.2743 - val_loss: 0.3915

## 모델 테스트하기

* 디코더의 동작 순서
1. 인코더에 입력 문장을 넣어 마지막 time step의 hidden, cell state를 얻는다.
2. 토큰인 '\t'를 디코더에 입력한다.
3. 이전 time step의 출력층의 예측 결과를 현재 time step의 입력으로 한다.
4. 3을 반복하다가 토큰인 '\n'가 예측되면 이를 중단한다.

In [23]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 53)]        0         
                                                                 
 lstm (LSTM)                 [(None, 256),             317440    
                              (None, 256),                       
                              (None, 256)]                       
                                                                 
Total params: 317,440
Trainable params: 317,440
Non-trainable params: 0
_________________________________________________________________


* 이전 time step의 hidden state를 저장하는 텐서
* 이전 time step의 cell state를 저장하는 텐서
* 이전 time step의 hidden state와 cell state를 하나의 변수에 저장

* decoder_states_inputs를 현재 time step의 초기 상태로 사용.
* 구체적인 동작 자체는 def decode_sequence()에 구현.
* 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.

In [24]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]

* 디코더의 출력층 재설계

In [28]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, 73)]   0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 lstm_1 (LSTM)                  [(None, None, 256),  337920      ['input_2[0][0]',                
                                 (None, 256),                     'input_3[0][0]',          

In [29]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

* 예측과정을 위한 함수 decode_sequence()를 구현합니다.
* 입력은 번역하고자 하는 문장의 정수 시퀀스입니다.
* 내부에는 인코더를 구현한 encoder_model이 있어서 이 모델에 번역하고자하는 문장의 정수 시퀀스인 'input_seq'를 입력하면
* encoder_model의 마지막 시점의 hidden state를 리턴합니다.

In [30]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

* 구현한 함수를 임의의 인덱스의 번역할 문장 샘플을 넣어서 테스트해보겠습니다.

In [31]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

1/1 [==============================] - 0s 14ms/step
-----------------------------------
입력 문장: Hi.
정답 문장:  Salut ! 
번역기가 번역한 문장:  salut ! 
1/1 [==============================] - 0s 13ms/step
-----------------------------------
입력 문장: Hello!
정답 문장:  Salut ! 
번역기가 번역한 문장:  aide-toi ! 
1/1 [==============================] - 0s 15ms/step
-----------------------------------
입력 문장: Hop in.
정답 문장:  Montez. 
번역기가 번역한 문장:  monte. 
1/1 [==============================] - 0s 14ms/step
-----------------------------------
입력 문장: Help me!
정답 문장:  Aide-moi ! 
번역기가 번역한 문장:  aide-moi ! 
1/1 [==============================] - 0s 16ms/step
-----------------------------------
입력 문장: Humor Tom.
정답 문장:  Mettez Tom de bonne humeur. 
번역기가 번역한 문장:  en avant m'instouche. 
